# Dette script laver oplæg til Genbrug og Brutto

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Definer variabler der bruges til at filtrere data med.

In [2]:
#Hvad skal resultatet være Alt (0), Flytning (1) eller Genanskaffelse (2) Special case (3)
res = 0

#Hvilke faser skal der filterers på første fase (1) anden fase (2) tredie fase/rokade (3) Special case (4)
etape = 4

#Ønskes det to resultat filer, en opdelt og en ikke opdelt eller kun en opdelt fil
toFiler = True

#Indflytningsdato for den givne fase (NB) md-dd-yyyy
f1 = pd.to_datetime('01-05-2023')
f2 = pd.to_datetime('01-09-2024')
f3 = pd.to_datetime('01-03-2025')

if etape == 1:
    NB = f1
    EN = 'Fase1'
elif etape == 2:
    NB = f2
    EN = 'Fase2'
elif etape == 3:
    NB = f3
    EN = 'Rokade'
elif etape == 4:
    NB = f3
    EN = 'Special'
    
#Maksimal alder (MA) i år ved indflytningen (hvis man vil have alt med (Brutto), sæt MA = 100)
if res == 0:
    MA = 100
else:
    MA = 10

#Dato for ælste udstyr der kan flyttes i NordBlokken (NordBlok Max Alder: NBMA)
NBMA = NB.replace(year=NB.year - MA)

#BagatelGrænse for pris (BG)
BG = 0 #3000 #100000 

#Skal der filterers på afdelinger (True/False)
filtAfd = True

#Skal der filterers på afsnit (True/False)
filtAfs = False

#Skal der filterers på Gråzone (True/False)  
grZ = True

#Skal der filterers på ekskluderede modeller 0=nej, 1=fjern, 2=find kun ekskluderede modeller
xModel = 0

#Skal der filterers på BIP(True/False)
BIP = False

#Skal der filterers på max af genanskaffelsespris og indkøbspris (True) ELLER kun indkøbspris (False)
pris = True




EJER = 'Bispebjerg og Frederiksberg Hospital'
#EJER = 'Nordsjællands Hospital'
if res == 0:
    filtrering =  'Alt'
elif res == 1:
    filtrering =  'Flytning'
elif res == 2:
    filtrering = 'Genanskaffelse'
elif res == 3:
    filtrering = 'Case'
    mrke = True
    toFiler = True

filNavn = EN + '_' + filtrering

In [3]:
FilterInput = pd.DataFrame({'Filter funktion':['Type filtrering','Etape/Fase','Ejer', 'Indflytnings dato', 'Udstyrets minimum alder [år]','Ælste dato [dato]', 'Bagatelgrænse i kr', 'Ekskluderede grupper'],
                           'Værdi': [filtrering,EN, EJER, NB, MA, NBMA, BG, 'nan']})

In [4]:
aar=[]
for i in range(4,24):
    aar.append(NB.replace(year=NB.year - i)) #Laver en liste med datoer i som kan bruges til filtrering eller til at
                                            #lave en oversigt over værdi af udstyr
#print(aar)

# Læs data
I der følgende indlæses data og arangeres i dataframes for at gøre det nemmere at analysere på fremad.

In [5]:
#filnavn(e) på excelark med Meudusa udtræk
filename = ['BBH.xlsx','FRH.xlsx'] 

#Er en fil med input til filtrering, som er uafhængig af Medusa udtræk
filterIndput = 'filterIndput.xlsx' 

#Hvis vi vil sortere afdelinger ud, kan gøres vha dette regneark.
AFD = 'Afdelinger.xlsx' 

df = pd.DataFrame()
for fil in filename:
    df = pd.read_excel(fil).append(df,sort=True) # Indlæs Medusa udtræk
#df.info()

In [6]:
if grZ:
    gz = pd.read_excel(filterIndput, sheet_name='Gråzone') # Indlæs Gråzone udstyr

if xModel:
    xmd = pd.read_excel(filterIndput, sheet_name='Xmodel') # Indlæs Ekskluderete modeller
    
if BIP:
    bip = pd.read_excel(filterIndput, sheet_name='BIP') # Indlæs BIP udstyr
    
if filtAfd:
    aFd = pd.read_excel(AFD, sheet_name='Afdelinger')
    if filtAfs:
        if etape == 1:
            F1 = pd.read_excel(AFD, sheet_name='Fase 1')
            fase = F1['Afsnit']
        elif etape == 2:
            F2 = pd.read_excel(AFD, sheet_name='Fase 2')
            fase = F2['Afsnit']
        elif etape == 3:
            Ro = pd.read_excel(AFD, sheet_name='Rokade')
            fase = Ro['Afsnit']
#afd.head()

In [7]:
#Tilpasninger
df['Indkøbt']= pd.to_datetime(df.Indkøbt)#konverter til datetime som Pandas kan regne på
if pris:
    df["Budget_Pris"] = df[["Indkøbspris","Genansk.pris"]].max(axis=1)# find den største værdi og sæt det ind i ny kollonde "Pris"
else:
    df['Budget_Pris'] = df['Indkøbspris']
    #df.head()

In [8]:
#Det næste laver en liste over afdelinger og skriver det i et regneark - så vi kan vælge afdelinger ud som vi vil se nærmere på
if 0: #behøver kun at køre én gang - ænder 0 til 1 
    Afdl = pd.unique(df['Ejer (overafdeling) 2'])
    #Afs = pd.unique(df['Afsnit (Placering)'])
    afdl = pd.DataFrame(Afdl)
    with pd.ExcelWriter('Afdelinger.xlsx') as writer:
        afdl.to_excel(writer,index = False)

# Så begynder filtereringen

In [9]:
x = df['Ejer (overafdeling) 2'].str.split(";", n = 1, expand = True)
df["Overafdeling"] =x[0]
df["Ejer"] = x[1]     

In [10]:
#Filterer på Ejer
fdf = df.loc[df.Ejer.str.contains(EJER)== True] 

#Filterer eventuelt på afdeling
if filtAfd:
    fdf = fdf.loc[fdf.Overafdeling.isin(aFd['Afdeling'])] 
if filtAfs:
    fdf = fdf.loc[fdf['Afsnit (Placering)'].isin(fase)] 
#fdf.head()

## Værdi for udstyr
For at kunne komme frem til et budget vil det være smart at kunne få et overblik over hvad fx alder på udstyret har af betydning for budgettet. Derfor laves der en samlet prissum pr alder. 

In [11]:
bg =[]
for i in range(len(aar)):
    bg.append(fdf.loc[fdf.Indkøbt >= aar[i], "Budget_Pris"].sum()) #Her bruges den filtrerede dataframe fdf, 
                                                            #for at få alle med skal man bruge dataframe df 
#plt.plot(aar,bg, '^') #simpel plot

oversigt = pd.DataFrame({'Årstal': aar, 'Værdi':bg}) # Lav oversigten til en dataframe
#oversigt.plot( x='Årstal',y='Værdi')
#oversigt

In [12]:
#Hvis der er brug for at fjerne eller finde specifikke modeller bruges denne celle
if xModel == 1:
    fdf = fdf.loc[~fdf.Model.isin(xmd.Model),:] # fjern ekskluderede modeller
elif xModel == 2:
    fdf = fdf.loc[fdf.Model.isin(xmd.Model),:] # find ekskluderede modeller
elif xModel == 0:
    pass
#print(len(ffdf))

In [13]:
#Filtrer på alder
if res ==1:
    fdf = fdf.loc[fdf.Indkøbt > NBMA,:] 
elif res == 2:
    fdf = fdf.loc[fdf.Indkøbt < NBMA,:]
elif res == 0 | res == 3:
    pass
    #fdf.head()

In [14]:
#Filtrer på pris
fdf =fdf.loc[fdf.Budget_Pris > BG,:] #maksimale pris bruges i sammenligningen

#fdf.head()
print(len(fdf))

3383


In [15]:
#Fjern BIP udstyr
if BIP:
    fdf = fdf.loc[~fdf.Gruppenavn.isin(bip.Gruppenavn)] 
    print(len(fdf))

In [16]:
#Fjern gråzone udstyr
if grZ:
    fdf = fdf.loc[~fdf.Gruppenavn.isin(gz.Gruppenavn)] 
    print(len(fdf))

2538


In [17]:
#Lidt oprydning
#OBS "Ejer (overafdeling)" kan have skiftet navn til "Afdeling (Ejer (overafdeling))" ret i excel!!!!
fdf = fdf.drop(['Indkøbspris', 'Genansk.pris', 'Afdeling (Ejer (overafdeling))'], axis=1)

In [18]:
#cols = fdf.columns.tolist()
#print(cols)

In [19]:
cols = ['App.nr', 'Gruppenavn', 'Fabrikat', 'Model', 'Indkøbt', 'Budget_Pris', 'Overafdeling', 'Afdeling (Placering)', 'Afsnit (Placering)', 'Ejer']
fdf = fdf[cols]

In [20]:
#Sorter på gruppenavn og indkøbsdato
fdf = fdf.sort_values(by=['Gruppenavn','Indkøbt'])
#fdf.info()

# Opdeling i afdelinger

In [21]:
Afd = pd.unique(fdf['Overafdeling'])
Afd.sort()

#print(Afd)

fdf['Overafdeling'] = fdf['Overafdeling'].str.slice(0,27)
afd = pd.unique(fdf['Overafdeling'])
afd.sort()

i=0
antal =[]
pris =[]
for x in afd:
    xls = fdf.loc[fdf['Overafdeling'].str.contains(x)==True]
    antal.append(len(xls)) # find antal udstyr i afdelingen
    pris.append(xls['Budget_Pris'].sum()) # find den samlede værdi af udstyret 
    i = i+len(xls)
   # print(x[0:28], "  ",len(xls))    
#print(i) #opsummetet antal
#antal
StatusArk =pd.DataFrame(list(zip(Afd, antal, pris)), columns =['Afdeling', 'Antal udstyr', 'Værdi'])
#StatusArk

In [22]:
Grupper = pd.DataFrame(fdf.groupby(['Overafdeling', 'Gruppenavn'])['Gruppenavn'].count())
#Grupper.unstack()

In [23]:
with pd.ExcelWriter(filNavn + '_Opdelt.xlsx') as writer:
    FilterInput.to_excel(writer, sheet_name='Opsumering',index = False)
    gz.to_excel(writer, sheet_name='Opsumering',startrow = 8,startcol = 1,index = False)
    StatusArk.to_excel(writer, sheet_name='Opsumering',startcol = 4, index = False)
    Grupper.to_excel(writer, sheet_name='Udstyrstyper i afdelingerne',index = True)
    Grupper.unstack().to_excel(writer, sheet_name='Udstyrstyper i afdelingerne', startcol = 5,index = True)
    
    for x in afd:
        xls = fdf.loc[fdf['Overafdeling'].str.contains(x)==True]
        if len(xls)>0:
            xls.to_excel(writer, sheet_name=x[0:27],index = False)

# Skriv det hele i et regneark

In [24]:
#print(f1)
#print(f2)
#print(f3)
mrke = False
if mrke:
    fdf = fdf.copy()
    f1 = f1.replace(year=f1.year - MA)
    f2 = f2.replace(year=f2.year - MA)
    f3 = f3.replace(year=f3.year - MA)
    fdf['Label'] = 'Ældre end ' + str(MA) + ' år' 
    
    #print(f1)
    #print(f2)
    #print(f3)
    
    filt1 = (fdf['Indkøbt'] > f1) #der kan sættes flere betingelser ind vha & eller |
    fdf.loc[filt1, 'Label'] = 'Fase 1'
    
    filt2 = (fdf['Indkøbt'] > f2) #der kan sættes flere betingelser ind vha & eller |
    fdf.loc[filt2, 'Label'] = 'Fase 2'
    
    filt3 = (fdf['Indkøbt'] > f3) #der kan sættes flere betingelser ind vha & eller |
    fdf.loc[filt3, 'Label'] = 'Fase 3'

In [25]:
if toFiler:
    with pd.ExcelWriter(filNavn + '.xlsx') as writer:
        fdf.to_excel(writer,index = False)